In [ ]:
%load_ext autoreload
%autoreload 2
%run -n main.py

# datasets

In [ ]:
# %run -n main.py
# for name in [SENT, TOKEN]:
#     path = join_path(DATA_DIR, name, DATASET)
#     !mkdir -p {path}

In [ ]:
# %run -n main.py
# for type in [TOKEN, SENT]:
#     log(type)
#     for name in [CORPORA, SYNTAG, GICRYA, RNC]:
#         paths = (
#             join_path(CORUS_DATA_DIR, _)
#             for _ in CORUS_FILES[name]
#         )
#         records = (
#             record
#             for path in paths
#             for record in DATASETS[name](path)
#         )
#         records = log_progress(records, desc=name)
#         partitions = PARSES[type][name](records)
#         lines = format_partitions(partitions)
#         path = join_path(DATA_DIR, type, DATASET, name + JL + GZ)
#         dump_gz_lines(lines, path)

# models

In [ ]:
# for type in [TOKEN, SENT]:
#     for name in MODELS[type]:
#         path = join_path(DATA_DIR, type, name)
#         !mkdir -p {path}

In [ ]:
# !pip install rusenttokenize
# !pip install segtok
# !pip install mosestokenizer
# !pip install razdel

# !pip install spacy
# !pip install https://github.com/aatimofeev/spacy_russian_tokenizer/archive/master.zip#egg=python-simhash
# !pip install https://github.com/Koziev/rutokenizer/archive/master.zip#egg=rutokenizer
# !pip install pymystem3

# !pip install nltk
# nltk.download('punkt')
# !wget https://raw.githubusercontent.com/mhq/train_punkt/master/russian.pickle -O ~/nltk_data/tokenizers/punkt/PY3/russian.pickle

# Texterra
# Можно ещё сравнивать с https://texterra.ispras.ru/products, но
# 1. она медленно работает, как минимум затраты на http
# 2. иногда кидает ошибку (возможно дело в английских предложениях)
# 3. качество немного выше segtok

# Polyglot
# реализует http://www.unicode.org/reports/tr29/

# Сорян, не смог установить. Дикие траблы с ICU
# brew install icu4c
# export ICU_VERSION=62.1
# export BASE=/usr/local/Cellar/icu4c/
# export PATH=$PATH:$BASE/$ICU_VERSION/bin
# export PYICU_INCLUDES=$BASE/$ICU_VERSION/include
# export PYICU_LFLAGS=-L$BASE/$ICU_VERSION/lib
# pip install pyicu polyglot

# Вроде установилось но 
# > from polyglot.text import Text
# > Text('...')
# Symbol not found: __ZNK6icu_6214Transliterator12getTargetSetERNS_10UnicodeSetE

In [ ]:
# for type in [SENT, TOKEN]:
#     log(type)
#     for model_name in MODELS[type]:
#         log(model_name)
#         model = MODELS[type][model_name]
#         if is_class(model):
#             model = model()
#         model = Timing(model)

#         for dataset_name in DATASETS:
#             path = join_path(DATA_DIR, type, DATASET, dataset_name + JL + GZ)
#             lines = load_gz_lines(path)
#             records = parse_partitions(lines)
#             records = head(records, 10000)
#             records = log_progress(records, desc=dataset_name)

#             records = (
#                 Partition.from_substrings(model(_.text))
#                 for _ in records
#             )
#             path = join_path(DATA_DIR, type, model_name, dataset_name + JL + GZ)
#             lines = format_partitions(records)
#             dump_gz_lines(lines, path)

#             path = join_path(DATA_DIR, STATS + JL)
#             record = [[type, model_name, dataset_name], model.time]
#             lines = format_jl([record])
#             append_lines(lines, path)

#             model.reset()

# score

In [ ]:
scores = {}
keys = [
    (type, model, dataset)
    for type in [TOKEN, SENT]
    for model in MODELS[type]
    for dataset in DATASETS
]
for type, model, dataset in log_progress(keys):
    path = join_path(DATA_DIR, type, DATASET, dataset + JL + GZ)
    lines = load_gz_lines(path)
    targets = parse_partitions(lines)

    path = join_path(DATA_DIR, type, model, dataset + JL + GZ)
    lines = load_gz_lines(path)
    preds = parse_partitions(lines)

    score = score_partitions(preds, targets)
    scores[type, model, dataset] = score

# report

In [ ]:
path = join_path(DATA_DIR, STATS + JL)
lines = load_lines(path)
items = parse_jl(lines)
times = {
    tuple(key): time
    for key, time in items
}

In [ ]:
table = report_table(scores, times, DATASETS, MODELS, TOKEN)
html = format_report(table, github=True)
HTML(html)

In [ ]:
table = report_table(scores, times, DATASETS, MODELS, SENT)
html = format_report(table, github=True)
HTML(html)